In [1]:
# Processing Pipeline for Lender and Borrower Data
#Import packages
import pandas as pd
import re
import logging

c:\Users\maroa\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# Load the original dataset
original_dataset = pd.read_csv("loandata_big.csv")

In [3]:
# Load dataset to a processing one for savekeeping original data
df_processing = original_dataset.copy()

In [4]:
# Drop rows where 'Lender Name / Role' is NaN
df_processing = df_processing.dropna(subset=["Lender Name / Role", "Maturity Date"])

# Replace '--' with NaN
df_processing["Tranche Amount (USD)"] = df_processing["Tranche Amount (USD)"].replace(
    "--", pd.NA
)
# Drop rows where 'Sector' is NaN in-place
df_processing.dropna(subset=["Tranche Amount (USD)"], inplace=True)
# Convert 'Value' column to float
df_processing["Tranche Amount (USD)"] = (
    df_processing["Tranche Amount (USD)"].str.replace(",", "").astype(float)
)


# Drop rows where Sector is 'Public Finance Activities'
df_processing.drop(
    df_processing[df_processing["Sector"] == "Public Finance Activities"].index,
    inplace=True,
)

In [5]:
# Convert columns to datetime with the specified format
df_processing['Maturity Date'] = pd.to_datetime(df_processing['Maturity Date'], format='%d-%m-%y').dt.normalize()
df_processing['Issue Date'] = pd.to_datetime(df_processing['Issue Date'], format='%d-%m-%y').dt.normalize()


In [6]:
df_processing

,Borrower Name,Lender Name / Role,Sector,RIC,Tranche Type,Issue Date,Maturity Date,Discounted Spread,Tranche Amount (USD),Tranche Amount,...,SIC,NAIC,Contributed Pricing Source,LIN,Sponsored,Price (Ask),Price (Bid),Price (Mid),Pricing Source,Yield (to 3 yrs)
3,GLASGOW SCHOOL PTE FINANCE INITIATIVE,"DEPFA Bank Plc:Participant, Halifax Plc:Bookru...",NaN,NaN,Term Loan,2001-05-08,2027-01-07,--,347422370.0,"277,250,000",...,Public Administration\Administration of Enviro...,Administration of Urban Planning and Community...,NaN,NaN,No,--,--,--,NaN,--
5,METRONET RAIL SSL FINANCE PLC,"BayernLB:Arranger, CIBC Inc:Mandated arranger,...","Passenger Transportation, Ground & Sea (NEC)",NaN,Revolver/Line >= 1 Yr.,2003-03-26,2030-03-25,--,144106664.0,"115,000,000",...,"Transportation, Communications, Electric, Gas,...",Other Financial Vehicles,NaN,NaN,No,--,--,--,NaN,--
6,METRONET RAIL BCV FINANCE PLC,"BayernLB:Arranger, CIBC Inc:Bookrunner, Deutsc...","Passenger Transportation, Ground & Sea (NEC)",NaN,Term Loan,2003-03-26,2030-03-25,--,413523471.0,"330,000,000",...,"Transportation, Communications, Electric, Gas,...",NaN,NaN,NaN,No,--,--,--,NaN,--
7,METRONET RAIL BCV FINANCE PLC,"BayernLB:Arranger, CIBC Inc:Bookrunner, Deutsc...","Passenger Transportation, Ground & Sea (NEC)",NaN,Revolver/Line >= 1 Yr.,2003-03-26,2030-03-25,--,144106664.0,"115,000,000",...,"Transportation, Communications, Electric, Gas,...",NaN,NaN,NaN,No,--,--,--,NaN,--
9,METRONET RAIL SSL FINANCE PLC,"BayernLB:Arranger, CIBC Inc:Bookrunner, Deutsc...","Passenger Transportation, Ground & Sea (NEC)",NaN,Term Loan,2003-03-26,2030-03-25,--,413523471.0,"330,000,000",...,"Transportation, Communications, Electric, Gas,...",Other Financial Vehicles,NaN,NaN,No,--,--,--,NaN,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6139,TURKIYE IS BANKASI AS,"ABC International Bank Plc:Arranger, ABSA Bank...",Banks (NEC),T001A5012TLK=R,Term Loan,2023-11-21,2024-11-22,-57.00,428138379.0,"411,000,000",...,"Finance, Insurance and Real Estate\Depository ...",Commercial Banking,LSTA REFINITIV PRICING,T001A5012TL0K,No,100.20,100.00,100.10,LSEG LPC Mark-to-Market Pricing - EMEA,--
6140,TURKIYE IS BANKASI AS,"ABC International Bank Plc:Arranger, ABSA Bank...",Banks (NEC),NaN,Term Loan,2023-11-21,2024-11-22,--,465000000.0,"465,000,000",...,"Finance, Insurance and Real Estate\Depository ...",Commercial Banking,NaN,NaN,No,--,--,--,NaN,--
6141,ARISTOTLE FUNDS SERIES TRUST,"BNP Paribas SA:Participant, Bank of Montreal:P...",Professional Information Services (NEC),NaN,364-Day Facility,2023-11-21,2024-11-19,--,225000000.0,"225,000,000",...,Services\Business Services\Miscellaneous Busin...,"All Other Professional, Scientific, and Techni...",NaN,NaN,No,--,--,--,NaN,--
6153,NASDAQ INC,"Bank of America Merrill Lynch:Admin agent, Cit...",Financial & Commodity Market Operators & Servi...,NaN,Revolver/Line >= 1 Yr.,2022-12-16,2027-12-16,--,345000000.0,"345,000,000",...,"Finance, Insurance and Real Estate\Security an...",Securities and Commodity Exchanges,NaN,NaN,No,--,--,--,NaN,--


In [7]:
print(df_processing['Maturity Date'].head())
print(df_processing['Maturity Date'].dtype)


3   2027-01-07
5   2030-03-25
6   2030-03-25
7   2030-03-25
9   2030-03-25
Name: Maturity Date, dtype: datetime64[ns]
datetime64[ns]


In [8]:

df_processing = df_processing[df_processing['Maturity Date'] >= pd.Timestamp('2024-01-01')]


In [9]:
# Select specific columns to keep
columns_to_keep = [
    "Borrower Name",
    "Lender Name / Role",
    "Issue Date",
    "Maturity Date",
    "Sector",
    "Tranche Amount (USD)",
    "Region",
    "Market of Issue",
    "Domicile",  # Domocile is Borrower, Region and Issue is Lender
    "Tranche Type",
    "Tranche Currency",
    "Seniority"
]
df_processing = df_processing[columns_to_keep]

In [10]:
print(df_processing.dtypes)


Borrower Name                   object
Lender Name / Role              object
Issue Date              datetime64[ns]
Maturity Date           datetime64[ns]
Sector                          object
Tranche Amount (USD)           float64
Region                          object
Market of Issue                 object
Domicile                        object
Tranche Type                    object
Tranche Currency                object
Seniority                       object
dtype: object


In [11]:
print(df_processing.columns)


Index(['Borrower Name', 'Lender Name / Role', 'Issue Date', 'Maturity Date',
       'Sector', 'Tranche Amount (USD)', 'Region', 'Market of Issue',
       'Domicile', 'Tranche Type', 'Tranche Currency', 'Seniority'],
      dtype='object')


In [12]:
# Function to filter and remove rows before any other processing
def filter_and_remove_rows(dataframe, column, names_to_remove, keywords):
    """
    Remove rows with specific names or containing specific keywords.
    """
    # Remove rows with explicit names
    dataframe = dataframe[~dataframe[column].isin(names_to_remove)]
    # Remove rows containing specific keywords
    pattern = "|".join(keywords)
    dataframe = dataframe[
        ~dataframe[column].str.contains(pattern, case=False, na=False)
    ]
    return dataframe


NAMES_TO_REMOVE = [
    # "INCORPORATED ADMINISTRATIVE AGENCY JAPAN HOUSING FINANCE AGENCY",
    # "JAPAN FINANCE ORGANIZATION FOR MUNICIPALITIES",
    "ANZ HOSPITALS PTY LTD",
    "GLASGOW SCHOOL PTE FINANCE INITIATIVE",
    "FRANKFURT SCHOOL OF FINANCE & MANAGEMENT GEMEINNUTZIGE GMBH",
    "HAMBURG ENERGIENETZE GMBH",
    "SCHIFFAHRTS-GESELLSCHAFT SEA CLOUD SPIRIT MBH",
    "DUBAI MULTI COMMODITIES CENTRE AUTHORITY",
    "JULIUS MEINL 1862 GMBH",
    "RING STATIONERY HOLDING GMBH",
    "ZIEGLER HOLDING GMBH",
    "ORASIS INDUSTRIES HOLDING GMBH",
    "PHILORO HOLDING GMBH",
    "SCHNEIDER HOLDING GMBH",
    "KAZ MINERALS FINANCE",
    "ONE NZ FINANCE LTD",
    "ASSETMARK FINANCIAL HOLDINGS INC",  # Because these do not have loan values
    "PAHAL FINANCIAL SERVICES PVT LTD",
    "CANARA BANK LTD",
]

KEYWORDS_TO_FILTER = ["MINISTRY", "DEPARTMENT"]

In [13]:
# Remove the rows with unnecessary data
# Filter rows
df_processing = filter_and_remove_rows(df_processing, 'Borrower Name', NAMES_TO_REMOVE, KEYWORDS_TO_FILTER)



In [14]:
# Function for lender consolidation
def normalize_lender_name(name):
    """
    Updated normalization function incorporating manual confirmations for new variations
    while retaining all previous logic.
    """
    # Convert to uppercase for consistent comparison
    name = name.upper()

    # Remove common suffixes and brackets
    suffixes = r"\b(SA|LTD|LLC|AG|INC|CO|PLC|CORP|CORPORATION|LIMITED|NV|NA|NATIONAL ASSOCIATION|SECURITIES|TRUST|BRANCH|GMBH|BANKING|FUNDING|&)\b"
    name = re.sub(suffixes, "", name)
    name = re.sub(r"\[.*?\]", "", name)  # Remove anything in square brackets

    # Compile regex patterns for performance
    patterns = {
        r"\bGOLDMAN SACHS\b": "GOLDMAN SACHS",
        r"\bBANK OF NOVA SCOTIA\b": "BANK OF NOVA SCOTIA",
        r"\bLLOYDS BANK\b|\bBANK OF SCOTLAND\b(?!\bROYAL BANK OF SCOTLAND\b)": "LLOYDS BANK",
        r"\b(MITSUBISHI UFJ FINANCIAL GROUP|MUFG|BANK OF TOKYO-MITSUBISHI|MITSUBISHI)\b": "MUFG",
        r"\bHUA NAN COMMERCIAL BANK\b": "HUA NAN COMMERCIAL BANK",
        r"\bTAIWAN BUSINESS BANK\b": "TAIWAN BUSINESS BANK",
        r"\bBANK OF COMMUNICATIONS\b": "BANK OF COMMUNICATIONS",
        r"\bSHANGHAI PUDONG DEVELOPMENT BANK\b": "SHANGHAI PUDONG DEVELOPMENT",
        r"\bKOREA DEVELOPMENT BANK\b": "KOREA DEVELOPMENT BANK",
        r"\bE SUN COMMERCIAL BANK\b": "E SUN COMMERCIAL BANK",
        r"\bTAISHIN INTERNATIONAL BANK\b": "TAISHIN INTERNATIONAL BANK",
        r"\bCIC CREDIT INDUSTRIEL ET COMMERCIAL\b": "CREDIT MUTUEL",
        r"\bCREDIT LYONNAIS\b": "CREDIT LYONNAIS",
        r"\bMONROE\b|\bMC CAPITAL\b": "MONROE CAPITAL",
        r"\bSOCIETE GENERALE\b": "SOCIETE GENERALE",
        r"\bMIZUHO\b": "MIZUHO",
        r"\bBANK OF THE PHILIPPINE\b": "BANK OF THE PHILIPPINE",
        r"\bBNP PARIBAS\b|\bFORTIS\b": "BNP PARIBAS",
        r"\bWELLS FARGO\b": "WELLS FARGO",
        r"\bANZ\b|\bAUSTRALIA & NEW ZEALAND\b": "ANZ",
        r"\bMEGA INTERNATIONAL COMMERCIAL BANK\b": "MEGA INTERNATIONAL COMMERCIAL BANK",
        r"\bNATIONAL BANK OF KUWAIT\b": "NATIONAL BANK OF KUWAIT",
        r"\bFIRST COMMERCIAL BANK\b": "FIRST COMMERCIAL BANK",
        r"\bSIEMENS\b": "SIEMENS BANK",
        r"\bCREDIT SUISSE\b": "CREDIT SUISSE",
        r"\bBANCO BILBAO VIZCAYA ARGENTARIA\b": "BBVA",
        r"\bBPCE\b|\bNATIXIS\b": "BPCE",
        r"\bCHIBA BANK\b": "CHIBA BANK",
        r"\bCHINA CONSTRUCTION BANK\b": "CHINA CONSTRUCTION BANK",
        r"\bCHINA MERCHANTS BANK\b": "CHINA MERCHANTS BANK",
        r"\bCHINA DEVELOPMENT BANK\b": "CHINA DEVELOPMENT BANK",
        r"\bCREDIT AGRICOLE\b": "CREDIT AGRICOLE",
        r"\bBANK OF AMERICA\b|\bMERRILL LYNCH\b": "BANK OF AMERICA",
        r"\bMINSHENG\b": "CHINA MINSHENG BANKING",
        r"\bING\b": "ING BANK",
        r"\bBANK OF CHINA\b": "BANK OF CHINA",
        r"\bSUMITOMO\b": "SMBC",
        r"\bJP MORGAN|JPMORGAN\b": "JP MORGAN CHASE",
        r"\bMORGAN STANLEY|MORGAN STANLEY BANK\b": "MORGAN STANLEY",
        r"\bHSBC\b": "HSBC",
        r"\bCITIBANK\b|\bCITI(GROUP)?\b": "CITI",
        r"\bDEUTSCHE BANK\b": "DEUTSCHE BANK",
        r"\bSTANDARD CHARTERED\b": "STANDARD CHARTERED",
        r"\bBARCLAYS\b": "BARCLAYS",
        r"\bCREDIT MUTUEL\b": "CREDIT MUTUEL",
        r"\bSANTANDER\b": " BANCO SANTANDER",
        r"\bBANCO POPULAR\b": "BANCO POPULAR",
        r"\bSTANDARD BANK OF SOUTH AFRICA\b": "STANDARD BANK GROUP",
        r"\bFUBON\b": "FUBON BANK",
        r"\bCITIC\b": "CITIC BANK",
        r"\bICBC\b": "IC BANK OF CHINA",
        r"\bBANK OF INDIA\b": "BANK OF INDIA",
        r"\bBANK OF BARODA\b": "BANK OF BARODA",
        r"\bBANK OF IRELAND\b": "BANK OF IRELAND",
        r"\bBANK OF MONTREAL\b": "BANK OF MONTREAL",
        r"\bUBS\b": "UBS",
        r"\bRBC\b": "RBC",
        r"\bTD BANK\b": "TD BANK",
        r"\bCOMMONWEALTH BANK OF AUSTRALIA\b": "COMMONWEALTH BANK OF AUSTRALIA",
        r"\bNATIONAL AUSTRALIA BANK\b|\bBANK OF NEW ZEALAND\b": "NATIONAL AUSTRALIA BANK",
        r"\bWESTPAC\b": "WESTPAC",
        r"MELLON|\bBANK OF NEW YORK\b": "BNY",
        r"\bCREDIT LYONNAIS\b": "CREDIT LYONNAIS",
        r"\bSTATE STREET BANK\b": "STATE STREET BANK",
        r"\bEXPORT DEVELOPMENT OF CANADA\b": "EXPORT DEVELOPMENT CANADA",
        r"\bBLACKSTONE\b": "BLACKSTONE", 
    }

    # Apply consolidations
    for pattern, replacement in patterns.items():
        if re.search(pattern, name):
            name = replacement

    # Strip extra spaces and return
    return re.sub(r"\s+", " ", name).strip()

In [15]:
# Split the "Lender Name / Role" column into lender names and roles
split_lenders_with_roles = (
    df_processing["Lender Name / Role"]
    .str.split(",", expand=True)
    .stack()
    .reset_index(level=1, drop=True)
    .str.split(":", n=1, expand=True)
    .rename(columns={0: "Original Lender Name", 1: "Role"})
)

# Merge the original dataset to retain all additional columns
processed_lenders_with_roles = (
    pd.DataFrame(split_lenders_with_roles)
    .merge(df_processing, left_index=True, right_index=True)
    .drop(
        columns=["Lender Name / Role"]
    )  # Optionally remove original column if not needed
)

# Apply the consolidation logic to normalize lender names
processed_lenders_with_roles["Consolidated Lender Name"] = processed_lenders_with_roles[
    "Original Lender Name"
].apply(normalize_lender_name)


# Save the processed dataset to a new CSV file
# processed_lenders_with_roles.to_csv('processed_dataset_with_roles_v3.csv', index=False)

In [16]:
# Processing Function for Borrower Data
def normalize_borrower_name(name, consolidations, general_terms):
    """
    Normalize borrower names with custom consolidation rules.
    """
    # Convert to uppercase
    name = name.upper()

    # Remove common suffixes
    suffixes = r"\b(SA|LTD|LLC|AG|INC|CO|PLC|CORP|NV|NA)\b"
    name = re.sub(suffixes, "", name)

    # Remove brackets and text inside
    name = re.sub(r"\[.*?\]|\(.*?\)", "", name)

    # Remove special characters and punctuation
    name = re.sub(r"[^\w\s]", "", name)

    # Specific consolidations using dictionary lookup
    for key, value in consolidations.items():
        if key in name:
            name = value

    # General replacements for substrings
    for term in general_terms:
        if term in name:
            name = term

    # Remove extra whitespace
    name = re.sub(r"\s+", " ", name).strip()

    return name


# Configuration for Borrower normalization
CONSOLIDATIONS = {
    "SUMITOMO": "SMBC",
    "OXFORD": "OXFORD FINANCE",
    "DEERPATH": "DEERPATH FUND",
    "MC CREDIT": "MONROE CAPITAL",
    "MONROE": "MONROE CAPITAL",
    "MITSUBISHI": "MUFG",
    "STANDARD BANK OF SOUTH AFRICA": "STANDARD BANK GROUP",
    "STANDARD CHARTERED": "STANDARD CHARTERED",
}

GENERAL_TERMS = [
    "BLACKSTONE",
    "MORGAN STANLEY",
    "GOLDMAN SACHS",
    "BLACKROCK",
    "VAKIF",
    "SCULPTOR REAL ESTATE",
    "ELAWAN FOTOVOLTAICA CAMPANARIO",
    "STOLT TANKERS FINANCE",
    "SPOWER FINANCE",
    "SOFTBANK",
    "DESRI SPRINGBOK",
    "BRIGHTWOOD CAPITAL FUND",
    "NEUBERGER BERMAN",
    "ACCIONA",
    "DOGGERBANK",
    "DAE FUNDING",
    "AUDAX MEZZANINE FUND",
    "AIC INCOME FUND",
    "WC FINANCING",
    "SDL II FINANCE",
    "MIDCAP FUNDING",
    "FOSUN INDUSTRIAL",
    "BLUE OWL",
    "AG REALTY VALUE FUND",
    "ESR FINANCE",
    "CREDIT AGRICOLE",
    "AVOLON AEROSPACE",
    "HPS SPECIALTY LOAN FUND",
    "VALMY FINANCEMENT",
    "KAYNE ANDERSON BDC FINANCING",
    "KANATLIAT AIRCRAFT FINANCE",
    "PLATINUM EQUITY SMALL CAP FUND",
    "HSCB",
    "MIZUHO",
    "CHINA DEVELOPMENT BANK",
    "SANTANDER ",
    "ROYAL BANK OF SCOTLAND",
    "UNION BANK OF INDIA",
    "BANK OF BARODA",
    "BANK OF IRELAND",
    "BANK OF MONTREAL",
    "BANK OF NEW YORK",
    "BANK OF NOVA SCOTIA",
    "BANK OF THE PHILIPPINE",
    "BANK OF CHINA",
    "SMBC",
    "JP MORGAN",
    "MORGAN STANLEY",
    "HSBC",
    "CITI",
    "STATE STREET BANK",
    "UBS"

]

In [17]:
# Normalize names
filtered_borrowers = processed_lenders_with_roles.copy()

filtered_borrowers["Normalized Borrower Name"] = filtered_borrowers[
    "Borrower Name"
].apply(lambda x: normalize_borrower_name(x, CONSOLIDATIONS, GENERAL_TERMS))


# Remove rows where 'Consolidated Lender Name' is NaN
filtered_borrowers = filtered_borrowers.dropna(subset=["Consolidated Lender Name"])
# This does not yet remove the NAs as there are no NAs yet in the dataset only when csv is called
# Fix this tmw

In [40]:
# Calculate frequency of names
#name_frequency = processed_lenders_with_roles['Consolidated Lender Name'].value_counts()
#bro_frequency = filtered_borrowers['Borrower Name'].value_counts()

# Display the frequency
#print(name_frequency.head(20))

In [18]:
filtered_borrowers = filtered_borrowers.reset_index()

In [50]:
df_dist = filtered_borrowers.copy()

In [51]:
# Define a dictionary for normalizing roles
role_mapping = {
    # Arranger roles
    'Lead arranger': 'Lead Arranger',
    'Mandated Lead arranger': 'Lead Arranger',
    'Arranger': 'Lead Arranger',
    'Co-arranger': 'Co-Arranger',
    'Joint arranger': 'Co-Arranger',
    'Co-lead arranger': 'Co-Arranger',
    'Senior arranger': 'Lead Arranger',
    'Lead bank': 'Lead Arranger',
    'Mandated arranger': 'Lead Arranger',
    # Agent roles
    'Admin agent': 'Agent',
    'Agent': 'Agent',
    'Syndication agent': 'Agent',
    'Facility agent': 'Agent',
    'Security agent': 'Agent',
    'Co-agent': 'Agent',
    'Documentation': 'Agent',  # or handle separately
    # Manager roles
    'Lead manager': 'Manager',
    'Manager': 'Manager',
    'Co-manager': 'Manager',
    'Joint lead manager': 'Manager',
    'Senior manager': 'Manager',
    # Lender roles
    'Lender': 'Participant',
    'Participant': 'Participant',
    
    'Sole lender': 'Sole Lender',   # might handle this as a special case
       # or Participant
    # Bookrunner could be mapped to Lead Arranger or kept separate
    'Bookrunner': 'Lead Arranger',
}

# A function to map each role to its standardized category
def normalize_role(role_str):
    # Lowercase everything for matching
    role_lower = role_str.strip().lower()
    # Try to match in the dictionary (using a small trick with a loop or direct dictionary lookups)
    for k, v in role_mapping.items():
        if k.lower() == role_lower:
            return v
    # If not found, default to 'Participant' or some 'Unknown' label
    return 'Participant'

# Create a new column for the standardized role
df_dist['StandardRole'] = df_dist['Role'].apply(normalize_role)

In [52]:

# Define base percentages for each role category
arrenger_perc_avg = sum([27.17, 31.34, 28, 31.37, 30.01, 24.98]) / len([27.17, 31.34, 28, 31.37, 30.01, 24.98])
agent_perc_avg = 11.679 #sum([11.679, 5.689]) / len([11.679, 5.689]) #alternative
participant_perc_avg = sum([20.847, 5.689]) / len([20.847, 5.689])

# Define a dictionary for base percentages
base_percentage = {
    'Lead Arranger': arrenger_perc_avg,
    'Co-Arranger': arrenger_perc_avg,   # Example if you want a special rate for co-arrangers
    'Co-Agent': agent_perc_avg,
    'Manager': agent_perc_avg,       # If you have manager roles
    'Participant': participant_perc_avg
}


In [53]:
def get_base_percentage(role_cat):
    # Sole Lender is handled differently
    if role_cat == 'Sole Lender':
        return 100  # We'll override logic anyway, but let's store 100
    return base_percentage.get(role_cat, participant_perc_avg)

# Depriciated because Sole lender needs to be handled separately
#df_dist['BasePct'] = df_dist['StandardRole'].apply(lambda x: base_percentage.get(x, participant_perc_avg))  # Default to participant

df_dist['BasePct'] = df_dist['StandardRole'].apply(get_base_percentage)

In [54]:
print("\nAssigned Base Percentage:")
df_dist


Assigned Base Percentage:


,index,Original Lender Name,Role,Borrower Name,Issue Date,Maturity Date,Sector,Tranche Amount (USD),Region,Market of Issue,Domicile,Tranche Type,Tranche Currency,Seniority,Consolidated Lender Name,Normalized Borrower Name,StandardRole,BasePct
0,5,BayernLB,Arranger,METRONET RAIL SSL FINANCE PLC,2003-03-26,2030-03-25,"Passenger Transportation, Ground & Sea (NEC)",144106664.0,EMEA,United Kingdom,United Kingdom,Revolver/Line >= 1 Yr.,British Pound,Senior,BAYERNLB,METRONET RAIL SSL FINANCE,Lead Arranger,28.811667
1,5,CIBC Inc,Mandated arranger,METRONET RAIL SSL FINANCE PLC,2003-03-26,2030-03-25,"Passenger Transportation, Ground & Sea (NEC)",144106664.0,EMEA,United Kingdom,United Kingdom,Revolver/Line >= 1 Yr.,British Pound,Senior,CIBC,METRONET RAIL SSL FINANCE,Lead Arranger,28.811667
2,5,Deutsche Bank AG,Mandated arranger,METRONET RAIL SSL FINANCE PLC,2003-03-26,2030-03-25,"Passenger Transportation, Ground & Sea (NEC)",144106664.0,EMEA,United Kingdom,United Kingdom,Revolver/Line >= 1 Yr.,British Pound,Senior,DEUTSCHE BANK,METRONET RAIL SSL FINANCE,Lead Arranger,28.811667
3,5,Royal Bank of Scotland Plc [US],Mandated arranger,METRONET RAIL SSL FINANCE PLC,2003-03-26,2030-03-25,"Passenger Transportation, Ground & Sea (NEC)",144106664.0,EMEA,United Kingdom,United Kingdom,Revolver/Line >= 1 Yr.,British Pound,Senior,LLOYDS BANK,METRONET RAIL SSL FINANCE,Lead Arranger,28.811667
4,5,Santander UK Plc [ex-Abbey National Plc],Mandated arranger,METRONET RAIL SSL FINANCE PLC,2003-03-26,2030-03-25,"Passenger Transportation, Ground & Sea (NEC)",144106664.0,EMEA,United Kingdom,United Kingdom,Revolver/Line >= 1 Yr.,British Pound,Senior,BANCO SANTANDER,METRONET RAIL SSL FINANCE,Lead Arranger,28.811667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34509,6154,Skandinaviska Enskilda Banken AB [SEB],Syndication agent,NASDAQ INC,2022-12-16,2027-12-16,Financial & Commodity Market Operators & Servi...,905000000.0,North America,United States,United States,Revolver/Line >= 1 Yr.,US Dollar,Senior,SKANDINAVISKA ENSKILDA BANKEN AB,NASDAQ,Agent,13.268000
34510,6154,Svenska Handelsbanken AB (publ),Documentation,NASDAQ INC,2022-12-16,2027-12-16,Financial & Commodity Market Operators & Servi...,905000000.0,North America,United States,United States,Revolver/Line >= 1 Yr.,US Dollar,Senior,SVENSKA HANDELSBANKEN AB (PUBL),NASDAQ,Agent,13.268000
34511,6154,TD Bank NA,Documentation,NASDAQ INC,2022-12-16,2027-12-16,Financial & Commodity Market Operators & Servi...,905000000.0,North America,United States,United States,Revolver/Line >= 1 Yr.,US Dollar,Senior,TD BANK,NASDAQ,Agent,13.268000
34512,6154,Toronto Dominion Bank,Participant,NASDAQ INC,2022-12-16,2027-12-16,Financial & Commodity Market Operators & Servi...,905000000.0,North America,United States,United States,Revolver/Line >= 1 Yr.,US Dollar,Senior,TORONTO DOMINION BANK,NASDAQ,Participant,13.268000


In [55]:
# -----------------------------------------------------------------------------
# 2) DEFINE A FUNCTION TO ASSIGN SHARES PER LOAN
# -----------------------------------------------------------------------------
def assign_shares_per_loan(group_df):
    """Given all rows for one LoanID, return a DataFrame with assigned shares."""
    
    # Check if there's a 'Sole Lender' in this subset
    if 'Sole Lender' in group_df['StandardRole'].unique():
        # 1) Identify the row with Sole Lender
        #    (The problem statement says there's exactly 1 in your edge cases.)
        mask_sole = group_df['StandardRole'] == 'Sole Lender'
        
        # 2) The sole lender gets 100% of the loan
        group_df.loc[mask_sole, 'ScaledPct'] = 100.0
        group_df.loc[mask_sole, 'AssignedLoanShare'] = (
            group_df.loc[mask_sole, 'Tranche Amount (USD)']
        )
        
        # 3) Everyone else gets 0%
        mask_others = ~mask_sole
        group_df.loc[mask_others, 'ScaledPct'] = 0.0
        group_df.loc[mask_others, 'AssignedLoanShare'] = 0.0
        
    else:
        # If no sole lender, do the normal scaling approach
        sum_base = group_df['BasePct'].sum()
        # Handle edge case if sum_base = 0
        if sum_base == 0:
            # If somehow no recognized roles, set them to 0 or distribute equally
            group_df['ScaledPct'] = 0
            group_df['AssignedLoanShare'] = 0
        else:
            scale_factor = 100.0 / sum_base
            group_df['ScaledPct'] = group_df['BasePct'] * scale_factor
            group_df['AssignedLoanShare'] = (
                group_df['ScaledPct'] / 100.0
            ) * group_df['Tranche Amount (USD)']  # This is the share of the loan amount
    
    # optional rounding
    group_df['AssignedLoanShare'] = group_df['AssignedLoanShare'].round(3)
    return group_df

In [56]:
# -----------------------------------------------------------------------------
# 3) APPLY THE FUNCTION GROUP-BY index
# -----------------------------------------------------------------------------
df_dist['ScaledPct'] = 0.0  # initialize
df_dist['AssignedLoanShare'] = 0.0

result_df = df_dist.groupby('index', group_keys=False).apply(assign_shares_per_loan)


In [57]:
# -----------------------------------------------------------------------------
# 4) SHOW THE FINAL RESULT
# -----------------------------------------------------------------------------
print("Final result with 'Sole Lender' logic:")
#print(result_df)

# Let's also confirm sums by LoanID
check_sums = result_df.groupby('index')['AssignedLoanShare'].sum()
print("\nSum of AssignedLoanShare by LoanID (should match TotalLoanValue):")
print(check_sums)

Final result with 'Sole Lender' logic:

Sum of AssignedLoanShare by LoanID (should match TotalLoanValue):
index
5       1.441067e+08
6       4.135235e+08
7       1.441067e+08
9       4.135235e+08
17      9.047434e+06
            ...     
6139    4.281384e+08
6140    4.650000e+08
6141    2.250000e+08
6153    3.450000e+08
6154    9.050000e+08
Name: AssignedLoanShare, Length: 5149, dtype: float64


In [58]:
# Select specific columns to keep
columns_to_keep = [
    "index",
    "Consolidated Lender Name",
    "Normalized Borrower Name",
    "StandardRole",
    "AssignedLoanShare",
    "Issue Date",
    "Maturity Date",
    "Sector",
    "Region",
    "Market of Issue",
    "Domicile",  # Domocile is Borrower, Region and Issue is Lender
    "Tranche Type",
    "Tranche Currency",
    "Seniority"
]
df_final = result_df[columns_to_keep]

In [59]:
df_final.rename(
    columns={
        "index": "LoanID",
        "Consolidated Lender Name": "Lender",
        "Normalized Borrower Name": "Borrower",
        "StandardRole": "Role",
        "Region": "LenderRegion",
        "Market of Issue": "LenderMarket",
        "Domicile": "BorrowerDomicile",
        "Tranche Type": "TrancheType",
        "Tranche Currency": "TrancheCurrency",
    },
    inplace=True,
)

<ipython-input-59-f7e64675f370>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.rename(


In [60]:
# Save the cleaned dataset to a new CSV file
df_final.to_csv('processed_loan_v2.csv', index=False, index_label= False)

In [44]:
unique_names = filtered_borrowers['Tranche Type'].unique()
print(unique_names)

['Revolver/Line >= 1 Yr.' 'Term Loan' 'Guarantee'
 'Standby Letter of Credit' 'Other Loan' 'VAT' 'Delay Draw Term Loan'
 'Export Credit' 'Schuldschein' 'Undisclosed' 'Construction Facility'
 'Term Loan B' 'Murabaha' 'Revolver/Term Loan' 'Term Loan A'
 'Acquisition Facility' 'Mortgage Facility' 'CAPEX Facility'
 'Mezzanine Tranche' 'Term Loan C' 'Bridge Loan' 'Trade Letter of Credit'
 '364-Day Facility' 'Term Loan D' 'Term Loan E' 'Bills Facility'
 'Debt Restructure' 'Lease' 'FRN (Loan-Style)' 'Wakala'
 'Revolver/Line < 1 Yr.']
